# [EX9] Movielens 영화 추천

#### * 목차
1. 데이터 준비와 전처리
2. 데이터 탐색
3. 모델 검증을 위한 사용자 초기 정보 세팅
4. 모델에 활용하기 위한 전처리(indexing)
5. CSR matrix 만들기
6. 모델 설계 및 학습
7. 사용자의 선호도 예측 비교
8. 사용자가 좋아하는 영화와 비슷한 영화 추천하기
9. 사용자가 좋아할만한 영화 추천하기
10. 결과

## 1. 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
# movie_id 기준으로 merge
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,count,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


## 2. 데이터 탐색

#### 1) ratings에 있는 유니크한 영화 개수    

In [7]:
ratings['movie_id'].nunique()

3628

#### 2) rating에 있는 유니크한 사용자 수  

In [8]:
ratings['user_id'].nunique()

6039

#### 3) 가장 인기 있는 영화 30개(인기순)   

In [9]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3. 모델 검증을 위한 사용자 초기 정보 세팅
내가 선호하는 영화를 5가지 골라서 rating에 추가

In [10]:
# 모델 검증을 위한 사용자 초기 정보 세팅
my_favorite = ['My Fair Lady (1964)', 'James and the Giant Peach (1996)', 'Platoon (1986)', 'Erin Brockovich (2000)', 'Heat (1995)']

my_list = pd.DataFrame({'user_id': ['7777']*5, 'title': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['7777']})['user_id'].any(): 
    ratings = ratings.append(my_list)                         

ratings.tail(10)      

,user_id,movie_id,count,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,7777,NaN,5,NaN,My Fair Lady (1964)
1,7777,NaN,5,NaN,James and the Giant Peach (1996)
2,7777,NaN,5,NaN,Platoon (1986)
3,7777,NaN,5,NaN,Erin Brockovich (2000)
4,7777,NaN,5,NaN,Heat (1995)


## 4.모델에 활용하기 위한 전처리(indexing)

In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# user, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [12]:
print(user_to_idx['7777'])    
print(movie_to_idx['Heat (1995)'])

6039
373


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,2,5,NaN,My Fair Lady (1964)
1,6039,1,5,NaN,James and the Giant Peach (1996)
2,6039,1030,5,NaN,Platoon (1986)
3,6039,3,5,NaN,Erin Brockovich (2000)


## 5. CSR matrix 만들기

In [14]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 6. 모델 설계 및 학습
Matrix Factorization 모델의 implicit 패키지

In [15]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
# 3. use_gpu : GPU를 사용할 것인지 
# 4. iterations : epochs와 같은 의미
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

## 7. 사용자의 선호도 예측 비교
내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보기

In [19]:
# MF 모델이 벡터를 만드는 코드
user, heat_moive = user_to_idx['7777'], movie_to_idx['Heat (1995)']
user_vector, heat_moive_vector = als_model.user_factors[user], als_model.item_factors[heat_moive]

print('슝=3')

슝=3


In [20]:
# 선호하는 영화의 백터의 곱
np.dot(user_vector,heat_moive_vector)

0.259127

In [21]:
# 선호하지 않는 영화의 백터의 곱
Chicken_movie = movie_to_idx['Chicken Run (2000)']
Chicken_movie_vector = als_model.item_factors[Chicken_movie]
np.dot(user_vector, Chicken_movie_vector)

0.09325943

## 8. 사용자가 좋아하는 영화와 비슷한 영화 추천하기

In [22]:
# (영화 id, 유사도) Tuple로 반환
favorite_movie = 'Heat (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(373, 1.0),
 (88, 0.72249156),
 (440, 0.614568),
 (377, 0.60868376),
 (242, 0.6021275),
 (278, 0.58578956),
 (840, 0.5783201),
 (220, 0.57570344),
 (987, 0.56598616),
 (289, 0.5569306),
 (136, 0.544068),
 (791, 0.5387277),
 (281, 0.5263432),
 (1788, 0.5243843),
 (1125, 0.52056223)]

In [23]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Heat (1995)',
 'Ronin (1998)',
 'True Romance (1993)',
 'In the Line of Fire (1993)',
 'Out of Sight (1998)',
 'Jackie Brown (1997)',
 'Simple Plan, A (1998)',
 'Seven (Se7en) (1995)',
 'Menace II Society (1993)',
 'Reservoir Dogs (1992)',
 'Desperado (1995)',
 'Negotiator, The (1998)',
 'Casino (1995)',
 'Hard 8 (a.k.a. Sydney, a.k.a. Hard Eight) (1996)',
 "Carlito's Way (1993)"]

In [24]:
# 사용자가 좋아하는 비슷한 영화 추천 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [25]:
get_similar_movie('Platoon (1986)')

['Platoon (1986)',
 'Full Metal Jacket (1987)',
 'Killing Fields, The (1984)',
 'Last Emperor, The (1987)',
 'Deer Hunter, The (1978)',
 'Patton (1970)',
 'Apocalypse Now (1979)',
 'Pink Floyd - The Wall (1982)',
 'Birdy (1984)',
 'Glory (1989)']

## 9. 사용자가 좋아할만한 영화 추천하기

In [26]:
user = user_to_idx['7777']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(14, 0.38389957),
 (391, 0.28898728),
 (45, 0.28068253),
 (890, 0.25112975),
 (354, 0.24708366),
 (4, 0.23063996),
 (685, 0.21806279),
 (43, 0.2084122),
 (984, 0.2076293),
 (10, 0.18732592),
 (330, 0.18615434),
 (1045, 0.1860107),
 (460, 0.18286881),
 (233, 0.18106328),
 (473, 0.17592551),
 (968, 0.16604038),
 (88, 0.16555986),
 (57, 0.16450518),
 (1084, 0.16300592),
 (468, 0.1621413)]

In [27]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Sound of Music, The (1965)',
 'Cider House Rules, The (1999)',
 'Mary Poppins (1964)',
 'Professional, The (a.k.a. Leon: The Professional) (1994)',
 'West Side Story (1961)',
 "Bug's Life, A (1998)",
 'Planet of the Apes (1968)',
 'Run Lola Run (Lola rennt) (1998)',
 'Dangerous Liaisons (1988)',
 'Beauty and the Beast (1991)',
 'Lion King, The (1994)',
 'Grumpy Old Men (1993)',
 'Boiler Room (2000)',
 'Usual Suspects, The (1995)',
 'Hurricane, The (1999)',
 'Last Emperor, The (1987)',
 'Ronin (1998)',
 'Untouchables, The (1987)',
 'Full Metal Jacket (1987)',
 'Magnolia (1999)']

## 10. 결과

* user_id : 7777
* 선호하는 Movies 
    1) My Fair Lady (1964)     
    2) James and the Giant Peach (1996)    
    3) Platoon (1986)    
    4) Erin Brockovich (2000)     
    5) Heat (1995) - vector : 0.259127    
* Heat(1995) 영화와 비슷한 영화 추천 : Ronin (1998) - vector 유사도 : 0.72249156  
* 좋아할만한 영화 추천 : Sound of Music, The (1965) - vector : 0.38389957